In [3]:
# !pip install transformers
# !pip install accelerate

In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
reviews = pd.read_csv("/content/drive/MyDrive/Transformers/Sentiment_Analysis/Movies_Dataset/movie_data.csv")
reviews.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [8]:
test_text = reviews.iloc[40000:, 0].tolist()
test_label = reviews.iloc[40000:, 1].values

In [9]:
model_name = "siebert/sentiment-roberta-large-english"

In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)

In [11]:
test_embedding = tokenizer(test_text, padding = True, truncation = True, max_length = 512)

In [12]:
class ReviewsDataset(Dataset):
    def __init__(self, embedding, label):
        self.embedding = embedding
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.embedding.items()}
        item["label"] = torch.tensor(self.label[idx])
        return item

In [13]:
test_dataset = ReviewsDataset(test_embedding, test_label)

In [14]:
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True, drop_last = True)

In [15]:
model = RobertaForSequenceClassification.from_pretrained(model_name)

In [16]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [17]:
def check_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct, total = 0, 0

        for batch in data_loader:

            embeddings = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(embeddings, attention_mask = attention_mask, labels = labels)
            loss, logits = outputs["loss"], outputs["logits"]

            _, predicted_labels = torch.max(logits, 1)
            correct += (predicted_labels == labels).sum()
            total += labels.size(0)

    return (correct.float() / total) * 100

In [18]:
model.eval()
model.to(device)

print(f'Test accuracy: {check_accuracy(model, test_loader, device):.2f}%')

Test accuracy: 95.15%
